In [18]:
from langchain_aws import ChatBedrockConverse
from langchain.schema import HumanMessage, SystemMessage
import boto3
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from opensearchpy import OpenSearch, RequestsHttpConnection
import os
import boto3
import json
import sys
from langchain.schema import BaseRetriever, Document
from typing import List, Optional, Dict, Tuple
from langchain.prompts.prompt import PromptTemplate
import uuid
from datetime import datetime
from botocore.exceptions import ClientError

In [19]:
bedrock_client = boto3.client(
    service_name='bedrock-runtime',
    region_name='us-east-1'  # replace with your region
)

In [20]:
# Another way to initialize with streaming
chat = ChatBedrockConverse(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    client=bedrock_client,
    temperature=0.7,
    max_tokens=2000
)


In [21]:
from langchain.memory import ConversationBufferMemory


In [22]:
#cpu의 메모리를 활용해서 conversation memory 사용. 추후 llm과 연동할수 있다
memory = ConversationBufferMemory()
memory


ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[]))

In [23]:
memory.save_context(
    inputs={
        "human": "안녕하세요, 비대면으로 은행 계좌를 원하는데 도와주실수 있나요?"
    },
    outputs={
        "ai": "안녕하세요! 계좌 개설을 원하신다니 기쁩니다. 먼저, 본인 인증을 위해 신분증을 준비해 주시겠어요?"
    },
)

In [24]:
print(memory.load_memory_variables({})["history"])


Human: 안녕하세요, 비대면으로 은행 계좌를 원하는데 도와주실수 있나요?
AI: 안녕하세요! 계좌 개설을 원하신다니 기쁩니다. 먼저, 본인 인증을 위해 신분증을 준비해 주시겠어요?


In [25]:
memory = ConversationBufferMemory(return_messages=True)

memory.save_context(
    inputs={
        "human": "안녕하세요, 비대면으로 은행 계좌를 개설하고 싶습니다. 어떻게 시작해야 하나요?"
    },
    outputs={
        "ai": "안녕하세요! 계좌 개설을 원하신다니 기쁩니다. 먼저, 본인 인증을 위해 신분증을 준비해 주시겠어요?"
    },
)

memory.save_context(
    inputs={"human": "네, 신분증을 준비했습니다. 이제 무엇을 해야 하나요?"},
    outputs={
        "ai": "감사합니다. 신분증 앞뒤를 명확하게 촬영하여 업로드해 주세요. 이후 본인 인증 절차를 진행하겠습니다."
    },
)

memory.save_context(
    inputs={"human": "사진을 업로드했습니다. 본인 인증은 어떻게 진행되나요?"},
    outputs={
        "ai": "업로드해 주신 사진을 확인했습니다. 이제 휴대폰을 통한 본인 인증을 진행해 주세요. 문자로 발송된 인증번호를 입력해 주시면 됩니다."
    },
)

In [26]:
print(memory.load_memory_variables({})["history"])


[HumanMessage(content='안녕하세요, 비대면으로 은행 계좌를 개설하고 싶습니다. 어떻게 시작해야 하나요?', additional_kwargs={}, response_metadata={}), AIMessage(content='안녕하세요! 계좌 개설을 원하신다니 기쁩니다. 먼저, 본인 인증을 위해 신분증을 준비해 주시겠어요?', additional_kwargs={}, response_metadata={}), HumanMessage(content='네, 신분증을 준비했습니다. 이제 무엇을 해야 하나요?', additional_kwargs={}, response_metadata={}), AIMessage(content='감사합니다. 신분증 앞뒤를 명확하게 촬영하여 업로드해 주세요. 이후 본인 인증 절차를 진행하겠습니다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='사진을 업로드했습니다. 본인 인증은 어떻게 진행되나요?', additional_kwargs={}, response_metadata={}), AIMessage(content='업로드해 주신 사진을 확인했습니다. 이제 휴대폰을 통한 본인 인증을 진행해 주세요. 문자로 발송된 인증번호를 입력해 주시면 됩니다.', additional_kwargs={}, response_metadata={})]


In [27]:
from langchain.chains import ConversationChain


# ConversationChain을 생성합니다.
conversation = ConversationChain(
    # ConversationBufferMemory를 사용합니다.
    llm=chat,
    memory=ConversationBufferMemory(),
)


In [28]:
response = conversation.predict(
    input="안녕하세요, 비대면으로 은행 계좌를 개설하고 싶습니다. 어떻게 시작해야 하나요?"
)
print(response)


안녕하세요. 비대면으로 은행 계좌를 개설하는 방법에 대해 알려드리겠습니다.

1. 해당 은행의 모바일 애플리케이션이나 인터넷 뱅킹 사이트에 접속하세요.

2. 신규 계좌 개설 메뉴를 찾아 절차를 진행하면 됩니다.

3. 개인정보 입력, 본인 인증, 약관 동의 등의 과정을 거치게 됩니다.

4. 본인 확인을 위해 영상통화나 휴대폰 인증 등의 절차가 있을 수 있습니다.

5. 계좌 개설이 완료되면 계좌번호와 관련 정보를 받게 됩니다.

전체적인 절차는 은행마다 조금씩 다를 수 있지만 대부분 모바일이나 인터넷을 통해 비대면으로 가능합니다. 문의사항이 더 있다면 말씀해주시기 바랍니다.


In [29]:
response = conversation.predict(
    input="이전 답변을 불렛포인트 형식으로 정리하여 알려주세요."
)
print(response)


물론입니다. 비대면으로 은행 계좌를 개설하는 절차를 불렛 포인트로 정리하면 다음과 같습니다:

• 해당 은행의 모바일 앱이나 인터넷뱅킹 사이트 접속
• 신규 계좌 개설 메뉴 찾아 절차 진행
• 개인정보 입력, 본인 인증, 약관 동의 등의 과정 거침
• 본인 확인을 위해 영상통화나 휴대폰 인증 절차 있을 수 있음
• 계좌 개설 완료 후 계좌번호와 관련 정보 수령

은행마다 세부 절차가 조금씩 다를 수 있지만, 대부분 모바일이나 인터넷을 통해 비대면으로 계좌 개설이 가능합니다.


In [30]:
print(conversation.memory.buffer)



Human: 안녕하세요, 비대면으로 은행 계좌를 개설하고 싶습니다. 어떻게 시작해야 하나요?
AI: 안녕하세요. 비대면으로 은행 계좌를 개설하는 방법에 대해 알려드리겠습니다.

1. 해당 은행의 모바일 애플리케이션이나 인터넷 뱅킹 사이트에 접속하세요.

2. 신규 계좌 개설 메뉴를 찾아 절차를 진행하면 됩니다.

3. 개인정보 입력, 본인 인증, 약관 동의 등의 과정을 거치게 됩니다.

4. 본인 확인을 위해 영상통화나 휴대폰 인증 등의 절차가 있을 수 있습니다.

5. 계좌 개설이 완료되면 계좌번호와 관련 정보를 받게 됩니다.

전체적인 절차는 은행마다 조금씩 다를 수 있지만 대부분 모바일이나 인터넷을 통해 비대면으로 가능합니다. 문의사항이 더 있다면 말씀해주시기 바랍니다.
Human: 이전 답변을 불렛포인트 형식으로 정리하여 알려주세요.
AI: 물론입니다. 비대면으로 은행 계좌를 개설하는 절차를 불렛 포인트로 정리하면 다음과 같습니다:

• 해당 은행의 모바일 앱이나 인터넷뱅킹 사이트 접속
• 신규 계좌 개설 메뉴 찾아 절차 진행
• 개인정보 입력, 본인 인증, 약관 동의 등의 과정 거침
• 본인 확인을 위해 영상통화나 휴대폰 인증 절차 있을 수 있음
• 계좌 개설 완료 후 계좌번호와 관련 정보 수령

은행마다 세부 절차가 조금씩 다를 수 있지만, 대부분 모바일이나 인터넷을 통해 비대면으로 계좌 개설이 가능합니다.


In [31]:
conversation.memory.clear()

